In [3]:
#Соединение с базой, получение курсора:

import psycopg2

# Параметры соединения
conn_string= "host='localhost' port=54320 dbname='my_database' user='root' password='postgres'" 

# Создаем соединение и курсор - это специальный объект который делает запросы и получает их результаты
with psycopg2.connect(conn_string) as conn, conn.cursor() as cursor:
    query = 'select * from lineitem limit 2' # запрос к БД
    cursor.execute(query) # выполнение запроса
    result = cursor.fetchall() # получение результата
    print(result)

[(39714, 133568, 1108, 1, Decimal('8.00'), Decimal('12812.48'), Decimal('0.05'), Decimal('0.08'), 'A', 'F', datetime.date(1994, 10, 25), datetime.date(1994, 12, 23), datetime.date(1994, 11, 23), 'NONE                     ', 'RAIL      ', 'ely regula'), (40097, 101287, 8818, 3, Decimal('16.00'), Decimal('20612.48'), Decimal('0.03'), Decimal('0.07'), 'R', 'F', datetime.date(1994, 4, 1), datetime.date(1994, 4, 2), datetime.date(1994, 4, 28), 'COLLECT COD              ', 'FOB       ', 'y quiet de')]


In [7]:
#Убедимся, что все таблицы были созданы и заполнены данными.
import psycopg2
with psycopg2.connect(conn_string) as conn, conn.cursor() as cursor:
    query = 'select * from nation limit 2' # запрос к БД
    cursor.execute(query) # выполнение запроса
    result = cursor.fetchall() # получение результата
    print(result)
    
with psycopg2.connect(conn_string) as conn, conn.cursor() as cursor:
    query = 'select * from orders limit 2' # запрос к БД
    cursor.execute(query) # выполнение запроса
    result = cursor.fetchall() # получение результата
    print(result)
    
with psycopg2.connect(conn_string) as conn, conn.cursor() as cursor:
    query = 'select * from part limit 2' # запрос к БД
    cursor.execute(query) # выполнение запроса
    result = cursor.fetchall() # получение результата
    print(result)
    
with psycopg2.connect(conn_string) as conn, conn.cursor() as cursor:
    query = 'select * from partsupp limit 2' # запрос к БД
    cursor.execute(query) # выполнение запроса
    result = cursor.fetchall() # получение результата
    print(result)

with psycopg2.connect(conn_string) as conn, conn.cursor() as cursor:
    query = 'select * from region limit 2' # запрос к БД
    cursor.execute(query) # выполнение запроса
    result = cursor.fetchall() # получение результата
    print(result)

with psycopg2.connect(conn_string) as conn, conn.cursor() as cursor:
    query = 'select * from supplier limit 2' # запрос к БД
    cursor.execute(query) # выполнение запроса
    result = cursor.fetchall() # получение результата
    print(result)

[(0, 'ALGERIA                  ', 0, ' haggle. carefully final deposits detect slyly agai'), (1, 'ARGENTINA                ', 1, 'al foxes promise slyly according to the regular accounts. bold requests alon')]
[(1, 36901, 'O', Decimal('173665.47'), datetime.date(1996, 1, 2), '5-LOW          ', 'Clerk#000000951', 0, 'nstructions sleep furiously among '), (2, 78002, 'O', Decimal('46929.18'), datetime.date(1996, 12, 1), '1-URGENT       ', 'Clerk#000000880', 0, ' foxes. pending accounts at the pending, silent asymptot')]
[(1, 'goldenrod lavender spring chocolate lace', 'Manufacturer#1           ', 'Brand#13  ', 'PROMO BURNISHED COPPER', 7, 'JUMBO PKG ', Decimal('901.00'), 'ly. slyly ironi'), (2, 'blush thistle blue yellow saddle', 'Manufacturer#1           ', 'Brand#13  ', 'LARGE BRUSHED BRASS', 1, 'LG CASE   ', Decimal('902.00'), 'lar accounts amo')]
[(1, 2, 3325, Decimal('771.64'), ', even theodolites. regular, final theodolites eat after the carefully pending foxes. furiously regular de

In [54]:
#Загружаем данные в наше хранилище:
import psycopg2

conn_string_s = "host='localhost' port=54320 dbname='my_database' user='root' password='postgres'" 
conn_s = psycopg2.connect(conn_string_s)
cursor_s = conn_s.cursor()

conn_string_t = "host='localhost' port=5433 dbname='my_database' user='root' password='postgres'" 
conn_t = psycopg2.connect(conn_string_t)
cursor_t = conn_t.cursor()

query_t = """SELECT column_name, column_default, data_type, character_maximum_length
FROM INFORMATION_SCHEMA.COLUMNS 
WHERE table_name = 'lineitem'"""
cursor_s.execute(query_t) # выполнение запроса
list_columns = cursor_s.fetchall() # получение результата

# print(list_columns)

q_create_table = 'CREATE TABLE lineitem ('

for col in list_columns:
    q_create_table += f'{col[0]} {col[2]}'
    if col[3]:
        q_create_table += f'({col[3]}), '
    else:
        q_create_table += ', '

q_create_table = q_create_table[0:-2] + ')' 

print(q_create_table)

q_drop_table = "DROP TABLE lineitem"

cursor_t.execute(q_drop_table)
conn_t.commit()

cursor_t.execute(q_create_table)
conn_t.commit()

cursor_t.execute('select * from lineitem')
result = cursor_t.fetchall() # получение результата
print(result)

#Дамп
q = "COPY lineitem TO STDOUT WITH DELIMITER ',' CSV HEADER;"
with open('lineitem_t.csv', 'w') as f:
    cursor_s.copy_expert(q, f)

#Вставка данных из дампа
q = "COPY lineitem FROM STDIN WITH DELIMITER ',' CSV HEADER;"
with open('lineitem_t.csv', 'r') as f:
    cursor_t.copy_expert(q, f)

    
#Смотрим, что зашло в новую таблицу и стравниваем с тем, что было в sourse:
cursor_t.execute('select * from lineitem limit 1')
result = cursor_t.fetchall() # получение результата
print(result)

cursor_s.execute('select * from lineitem limit 1')
result = cursor_s.fetchall() # получение результата
print(result)

CREATE TABLE lineitem (l_orderkey integer, l_partkey integer, l_suppkey integer, l_linenumber integer, l_quantity numeric, l_extendedprice numeric, l_discount numeric, l_tax numeric, l_returnflag character(1), l_linestatus character(1), l_shipdate date, l_commitdate date, l_receiptdate date, l_shipinstruct character(25), l_shipmode character(10), l_comment character varying(44))
[]
[(39714, 133568, 1108, 1, Decimal('8.00'), Decimal('12812.48'), Decimal('0.05'), Decimal('0.08'), 'A', 'F', datetime.date(1994, 10, 25), datetime.date(1994, 12, 23), datetime.date(1994, 11, 23), 'NONE                     ', 'RAIL      ', 'ely regula')]
[(39714, 133568, 1108, 1, Decimal('8.00'), Decimal('12812.48'), Decimal('0.05'), Decimal('0.08'), 'A', 'F', datetime.date(1994, 10, 25), datetime.date(1994, 12, 23), datetime.date(1994, 11, 23), 'NONE                     ', 'RAIL      ', 'ely regula')]


In [ ]:
#Загружаем данные в наше хранилище:
import psycopg2

conn_string_s = "host='localhost' port=54320 dbname='my_database' user='root' password='postgres'" 
conn_s = psycopg2.connect(conn_string_s)
cursor_s = conn_s.cursor()

conn_string_t = "host='localhost' port=5433 dbname='my_database' user='root' password='postgres'" 
conn_t = psycopg2.connect(conn_string_t)
cursor_t = conn_t.cursor()


def create_table_in_target(table_name):
    query_t = f"SELECT column_name, column_default, data_type, character_maximum_length FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = '{table_name}'"
    cursor_s.execute(query_t) # выполнение запроса
    list_columns = cursor_s.fetchall() # получение результата

    # print(list_columns)

    q_create_table = f'CREATE TABLE {table_name} ('

    for col in list_columns:
        q_create_table += f'{col[0]} {col[2]}'
        if col[3]:
            q_create_table += f'({col[3]}), '
        else:
            q_create_table += ', '

    q_create_table = q_create_table[0:-2] + ')' 

    print(q_create_table)

    q_drop_table = f"DROP TABLE IF EXISTS {table_name}"

    cursor_t.execute(q_drop_table)
    conn_t.commit()

    cursor_t.execute(q_create_table)
    conn_t.commit()

    cursor_t.execute(f'select * from {table_name}')
    result = cursor_t.fetchall() # получение результата
    print(result)

    #Дамп
    q = f"COPY {table_name} TO STDOUT WITH DELIMITER ',' CSV HEADER;"
    with open('res_f.csv', 'w') as f:
        cursor_s.copy_expert(q, f)

    #Вставка данных из дампа
    q = f"COPY {table_name} FROM STDIN WITH DELIMITER ',' CSV HEADER;"
    with open('res_f.csv', 'r') as f:
        cursor_t.copy_expert(q, f)


    #Смотрим, что зашло в новую таблицу и стравниваем с тем, что было в sourse:
    cursor_t.execute(f'select * from {table_name} limit 1')
    result = cursor_t.fetchall() # получение результата
    print(result)

    cursor_s.execute(f'select * from {table_name} limit 1')
    result = cursor_s.fetchall() # получение результата
    print(result)

t_n = ['lineitem', 'nation', 'orders', 'part', 'partsupp', 'region', 'supplier']

for t in t_n:
    create_table_in_target(t)

CREATE TABLE lineitem (l_orderkey integer, l_partkey integer, l_suppkey integer, l_linenumber integer, l_quantity numeric, l_extendedprice numeric, l_discount numeric, l_tax numeric, l_returnflag character(1), l_linestatus character(1), l_shipdate date, l_commitdate date, l_receiptdate date, l_shipinstruct character(25), l_shipmode character(10), l_comment character varying(44))
